# Student's t-test for significantly different variances in Python

In [1]:
import numpy as np
import timeit
import sys
import os
from subprocess import call, check_output

# Import Python functions
sys.path.append('../../avevar')
from avevar import avevar

sys.path.append('../lib_stat')
from lib_stat import betai

# Compile fortran source code (using a Python script)
os.system('python ../lib_stat/compile_fortran.py');

Data input:

In [2]:
data1 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
data2 = np.array([1.1, 2, 3, 4, 5, 6, 7, 8, 9])

Function definition:

In [3]:
def tutest(data1, data2):
    # Given the arrays "data1" and "data2", returns the
    # Student's "t" and its significance as "prob".
    # Data are assumed to be drawn from populations with
    # the same true variance.
    
    # Small values of "prob" indicates that the arrays 
    # have different means.
    n1 = len(data1)
    n2 = len(data2)
    
    ave1, var1 = avevar(data1)
    ave2, var2 = avevar(data2)

    # print(ave1, ave2)
    # print(var1, var2)
    
    # Student's t
    t = (ave1 - ave2) / np.sqrt(var1/n1 + var2/n2)

    df = (var1/n1 + var2/n2)**2 / ((var1/n1)**2 / (n1 - 1) + (var2/n2)**2 / (n2 - 1))
    # significance
    prob = betai(0.5 * df, 0.5, df/(df + t**2))
    
    return t, prob

#### Calling the Python tutest function:

In [4]:
t, prob = tutest(data1, data2)

print(t, prob)

-0.008635140708713388 0.9932169972438553


#### Calling the Fortran tutest function:

In [5]:
fortranOutput = check_output('../lib_stat/fmain_tutest').decode("utf-8").replace('\n','').split(' ')
[float(i) for i in fortranOutput if i]

[-0.008635140708713388, 0.9932169972438553]

# Timing against Fortran:

Test function in **Python** and timing:

In [6]:
def speed_test():
    for i in range(1000):
        tutest(data1, data2)
        
%timeit speed_test()

15.7 ms ± 72.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Timing in **Fortran**:

In [7]:
%timeit call(['../lib_stat/fspeed_tutest'])

5.45 ms ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
